# Raw Data Prep

## Libraries

In [1]:
import numpy as np
import pandas as pd
import xlrd
import os

In [2]:
import sys
np.set_printoptions(threshold=np.inf)

## Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#os.listdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
os.listdir(".")

In [ ]:
housePath = "/content/drive/My Drive/Colab Notebooks/house_prices.xlsx"

# Raw Jobs Data into dataframe


In [3]:
#xl2 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/boise_msa_jobs_2000_to_2018.xlsx")
xl2 = pd.ExcelFile("./Data/boise_msa_jobs_2000_to_2018.xlsx")
rawJobs = xl2.parse("raw")
rawJobs.to_pickle("rawJobs.pkl")

In [ ]:
rawJobs.group.unique()

### From rawJobs.pkl to metro_jobs from pickle

In [4]:
# open the pickle and turn it into a dataframe
rawJobs = pd.read_pickle("rawJobs.pkl")
metroJobs = rawJobs.copy()

# Splitting Strings
metroJobs['Occ_Cat_Code'], metroJobs['Occ_Sub_Cat_Code'] = metroJobs['occ_code'].str.split('-',1).str


# fixing the ['group'] column
metroJobs['group'] = metroJobs['group'].replace(np.nan,'detailed', regex=True)
metroJobs['occ_titl'] = metroJobs['occ_titl'].replace('Industry Total', 'All Occupations', regex = True)
metroJobs.loc[metroJobs.occ_titl == 'All Occupations', 'group'] = 'total'




# Assigning New Column Names
metroJobs = metroJobs.rename(columns ={'occ_titl': 'Occ_Title', 'group':'Cat_Type',
                                        'tot_emp': 'Emp_Count', 'h_mean':'Hr_Wage_Mean',
                                        'a_mean':'An_Wage_Mean', 'h_median': 'Hr_Median',
                                        'a_median':'An_Median','year':'Year'})



metroJobs['Year'] = metroJobs['Year'].astype(str)
metroJobs['Year'] = metroJobs['Year'].astype('datetime64[Y]')



# Assigning New Data Types
float_list = ['Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']

# Cleaning Column Strings for Data Typing
metroJobs[float_list] = metroJobs[float_list].replace('\W','',regex=True)

# Assigning new Data Types
#         need to use floats becuase int cannot handle missing
metroJobs = metroJobs.astype({ 'Cat_Type' : 'category',
                               'Occ_Cat_Code':'category','Occ_Sub_Cat_Code': 'category'})

metroJobs[float_list] = metroJobs[float_list].apply(pd.to_numeric, errors = 'coerce')


# Dropped Columns
metroJobs = metroJobs.drop(['prim_state','area','area_name','occ_code',
                              'emp_prse','mean_prse','h_wpct10','h_wpct25',
                             'h_wpct75','h_wpct90','a_wpct10','a_wpct25',
                             'a_wpct75','a_wpct90'], axis = 1)

# Assigning New Order to the Columns
metroJobs = metroJobs[['Year','Occ_Cat_Code','Occ_Sub_Cat_Code','Occ_Title','Cat_Type',
                        'Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']]

metroJobs['Occ_Title'] = metroJobs['Occ_Title'].str.title()
metroJobs['Emp_Count'] = metroJobs['Emp_Count'].fillna(0)
metroJobs['An_Wage_Mean'] = metroJobs['An_Wage_Mean'].fillna(0)



metroJobs.to_pickle('metroJobs.pkl')
metroJobs.head(5)

,Year,Occ_Cat_Code,Occ_Sub_Cat_Code,Occ_Title,Cat_Type,Emp_Count,Hr_Wage_Mean,An_Wage_Mean,Hr_Median,An_Median
0,2000-01-01,11,0000,Management Occupations,major,15790.0,2950.0,61360.0,2668.0,55490.0
1,2000-01-01,11,1011,Chief Executives,detailed,1100.0,4802.0,99870.0,4889.0,101680.0
2,2000-01-01,11,1021,General And Operations Managers,detailed,5180.0,2761.0,57430.0,2392.0,49750.0
3,2000-01-01,11,2011,Advertising And Promotions Managers,detailed,220.0,2020.0,42030.0,1903.0,39590.0
4,2000-01-01,11,2021,Marketing Managers,detailed,520.0,3238.0,67350.0,2708.0,56330.0


In [5]:
# jobsTest = metroJobs[metroJobs['Emp_Count'] > 200000]
# jobsTest

metroJobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8947 entries, 0 to 8946
Data columns (total 10 columns):
Year                8947 non-null datetime64[ns]
Occ_Cat_Code        8947 non-null category
Occ_Sub_Cat_Code    8947 non-null category
Occ_Title           8947 non-null object
Cat_Type            8947 non-null category
Emp_Count           8947 non-null float64
Hr_Wage_Mean        8524 non-null float64
An_Wage_Mean        8947 non-null float64
Hr_Median           8483 non-null float64
An_Median           8813 non-null float64
dtypes: category(3), datetime64[ns](1), float64(5), object(1)
memory usage: 537.6+ KB


# Clean jobs for empolyee count for job type using major job categories

In [6]:
metroJobs = pd.read_pickle('metroJobs.pkl')

# start filtering to make df with Occ_Title as cols and the value be Emp_Count for each year.

# 0000 is the sub_cat # for the head of Cat
jobsF = metroJobs.copy()
jobsF = jobsF.loc[jobsF['Cat_Type'] != 'detailed']


jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Services','Service', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Practitioners','Practitioner', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace(' Mathematical Science Occupations',' Mathematical Occupations', regex = True)


jobsClean = jobsF.copy()
jobsClean.to_pickle('jobsClean.pkl')
jobsClean.head()


,Year,Occ_Cat_Code,Occ_Sub_Cat_Code,Occ_Title,Cat_Type,Emp_Count,Hr_Wage_Mean,An_Wage_Mean,Hr_Median,An_Median
0,2000-01-01,11,0000,Management Occupations,major,15790.0,2950.0,61360.0,2668.0,55490.0
24,2000-01-01,13,0000,Business And Financial Operations Occupations,major,9560.0,2150.0,44710.0,1942.0,40390.0
45,2000-01-01,15,0000,Computer And Mathematical Occupations,major,6020.0,2577.0,53600.0,2377.0,49440.0
54,2000-01-01,17,0000,Architecture And Engineering Occupations,major,12860.0,2178.0,45300.0,2062.0,42900.0
73,2000-01-01,19,0000,"Life, Physical, And Social Science Occupations",major,2650.0,1714.0,35650.0,1540.0,32040.0


In [ ]:
#jobsClean[jobsClean['Cat_Type'] == 'major'].count()
#jobsClean.shape
#metroJobs.Cat_Type.nunique()

## Jobs by Industry, employee count

In [7]:
jobsF1 = pd.read_pickle('jobsClean.pkl')
jobsF1 = jobsF.drop(['Occ_Cat_Code', 'Occ_Sub_Cat_Code', 'Cat_Type'], axis =1)
jobsF1 = jobsF1.iloc[:,0:3]

jobsP1 = pd.pivot(jobsF1, values = 'Emp_Count', index = 'Year', columns = 'Occ_Title')
jobsP1 = jobsP1.reset_index()

# Gives the wrong answer, 2018 should be 706140
jobsP1['Job_Manual_Count'] = jobsP1.iloc[:,2:].sum(axis = 1)

#df['TAV_Ch'] = df.groupby('Parcel')['TAV'].diff()

jobsP1.at[0,'All Occupations'] = 229780
jobsP1.to_pickle('jobTypes.pkl')
jobsP1

Occ_Title,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,Legal Occupations,"Life, Physical, And Social Science Occupations",Management Occupations,Office And Administrative Support Occupations,Personal Care And Service Occupations,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations,Job_Manual_Count
0,2000-01-01,229780.0,12860.0,2730.0,7570.0,9560.0,2840.0,6020.0,13650.0,10950.0,...,1670.0,2650.0,15790.0,38160.0,2830.0,19040.0,3850.0,21960.0,14930.0,229780.0
1,2001-01-01,222100.0,11400.0,2480.0,7610.0,9030.0,2790.0,5670.0,13620.0,9690.0,...,1660.0,2080.0,16050.0,37380.0,2710.0,16470.0,3830.0,21410.0,14250.0,222100.0
2,2002-01-01,225820.0,11880.0,2590.0,7950.0,9380.0,2990.0,6190.0,11790.0,NaN,...,1730.0,2220.0,15800.0,39150.0,2990.0,15580.0,4120.0,22310.0,14620.0,216720.0
3,2003-01-01,228350.0,7010.0,2540.0,7780.0,9120.0,3270.0,5630.0,12100.0,NaN,...,1690.0,2440.0,15580.0,39690.0,3220.0,17330.0,4380.0,24540.0,14620.0,215740.0
4,2004-01-01,232950.0,7310.0,3100.0,7960.0,9630.0,3360.0,6700.0,12010.0,11440.0,...,1860.0,2290.0,17130.0,40830.0,3130.0,15520.0,5060.0,24210.0,13990.0,232950.0
5,2005-01-01,254330.0,7720.0,2860.0,7620.0,10310.0,3150.0,7090.0,0.0,11910.0,...,2040.0,2880.0,19450.0,44340.0,4380.0,18010.0,4790.0,24940.0,14880.0,238590.0
6,2006-01-01,268600.0,8950.0,3310.0,8490.0,11640.0,3500.0,6970.0,18680.0,11860.0,...,1940.0,2540.0,21200.0,46910.0,5150.0,18100.0,4330.0,26200.0,16450.0,268610.0
7,2007-01-01,284160.0,8480.0,4430.0,9100.0,11490.0,4160.0,7330.0,19070.0,12040.0,...,1780.0,2830.0,20000.0,47860.0,6090.0,20100.0,4830.0,30030.0,19430.0,284130.0
8,2008-01-01,286080.0,8660.0,4630.0,8740.0,11440.0,5480.0,6920.0,15340.0,12030.0,...,2030.0,3300.0,20120.0,47380.0,5930.0,19510.0,5730.0,31410.0,18600.0,286070.0
9,2009-01-01,264010.0,7950.0,4960.0,8340.0,10680.0,5480.0,7580.0,12400.0,12590.0,...,2270.0,3110.0,18200.0,43710.0,6040.0,14490.0,5410.0,28840.0,17620.0,263990.0


In [17]:
jobsP1.shape

(19, 25)

## Clean jobs for empolyee count for Salery Range not using major job categories

In [ ]:
metroJobs = pd.read_pickle('metroJobs.pkl')

jobsF2 = metroJobs.copy()

jobsF2 = jobsF2.loc[jobsF2['Cat_Type'] == 'detailed']


jobsF2 = jobsF2[['Year', 'Emp_Count' ,'An_Wage_Mean']]
#jobsF2 = jobsF2[np.isfinite(jobsF2['An_Wage_Mean'])]
#jobsF2['Emp_Count'] = jobsF2['Emp_Count'].fillna(0)
#jobsF2['']fillna(0)

jobsF2.head()

random code for searching for errors

In [ ]:
x = pd.read_pickle('metroJobs.pkl')
x = metroJobs.loc[metroJobs['Year'] == 2016]
x = x.loc[x['An_Wage_Mean'] < 200000]
#y = jobsF2.loc[jobsF2['An_Wage_Mean'] == np.nan]
#z = metroJobs.copy()
x.sort_values(['An_Wage_Mean'], ascending = False)
#jobsF2.isna().sum()
#jobsF2.shape
#jobsF2.Cat_Type.unique()

## Jobs by Salery Range, empolyee count

In [11]:
df = pd.read_pickle('metroJobs.pkl')

df = df.loc[df['Cat_Type'] == 'detailed']
df = df[['Year', 'Emp_Count' ,'An_Wage_Mean']]

jobsSalery = df.groupby([df.Year,pd.cut(df.An_Wage_Mean,[10000,40000,70000,100000,150000,200000,300000]).astype(str)])[['Emp_Count']].sum().reset_index()

jobsSalery = pd.pivot(jobsSalery, values = 'Emp_Count', index = 'Year', columns = 'An_Wage_Mean')
jobsSalery = jobsSalery.reset_index()
jobsSalery = jobsSalery.replace(np.nan,0)

jobsSalery = jobsSalery[['Year','(10000.0, 40000.0]','(40000.0, 70000.0]','(70000.0, 100000.0]',
          '(100000.0, 150000.0]', '(150000.0, 200000.0]', '(200000.0, 300000.0]' ]]

jobsSalery.to_pickle('jobSalery.pkl')
jobsSalery

An_Wage_Mean,Year,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,150080.0,34980.0,3710.0,0.0,0.0,0.0
1,2001-01-01,135860.0,45570.0,4520.0,0.0,0.0,0.0
2,2002-01-01,129450.0,40330.0,5850.0,1100.0,0.0,0.0
3,2003-01-01,132290.0,40650.0,4960.0,1160.0,0.0,0.0
4,2004-01-01,143120.0,44460.0,5710.0,2030.0,0.0,0.0
5,2005-01-01,147750.0,53520.0,4010.0,3070.0,220.0,0.0
6,2006-01-01,159950.0,50600.0,9800.0,3970.0,200.0,0.0
7,2007-01-01,167110.0,60680.0,20230.0,3950.0,240.0,0.0
8,2008-01-01,162520.0,64660.0,22890.0,4680.0,250.0,0.0
9,2009-01-01,146070.0,65420.0,23110.0,1280.0,230.0,590.0


In [12]:
jobsSalery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
Year                    19 non-null datetime64[ns]
(10000.0, 40000.0]      19 non-null float64
(40000.0, 70000.0]      19 non-null float64
(70000.0, 100000.0]     19 non-null float64
(100000.0, 150000.0]    19 non-null float64
(150000.0, 200000.0]    19 non-null float64
(200000.0, 300000.0]    19 non-null float64
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.1 KB


In [16]:
job1 = pd.read_pickle('jobTypes.pkl')
job2 = pd.read_pickle('jobSalery.pkl')

jobFeatures = pd.merge(job1, job2, on = 'Year', how = 'left')
jobFeatures = jobFeatures.drop(['Job_Manual_Count'], axis = 1)

jobFeatures.to_pickle('jobFeatures.pkl')
jobFeatures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 30 columns):
Year                                                          19 non-null datetime64[ns]
All Occupations                                               19 non-null float64
Architecture And Engineering Occupations                      19 non-null float64
Arts, Design, Entertainment, Sports, And Media Occupations    19 non-null float64
Building And Grounds Cleaning And Maintenance Occupations     19 non-null float64
Business And Financial Operations Occupations                 19 non-null float64
Community And Social Service Occupations                      19 non-null float64
Computer And Mathematical Occupations                         19 non-null float64
Construction And Extraction Occupations                       19 non-null float64
Education, Training, And Library Occupations                  17 non-null float64
Farming, Fishing, And Forestry Occupations                    19 non-n